Importing Libraries


In [33]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import pandas as pd
import os
import time
from collections import deque
from sklearn.preprocessing import LabelEncoder
from threading import Thread
import queue

# ============================================
# 📱 CAMERA SOURCE CONFIGURATION
# ============================================
# Choose your camera source:
# - "webcam"    : Use built-in/USB webcam (default)
# - "phone_ip"  : Use phone camera via IP Webcam app (Android)
# - "droidcam"  : Use DroidCam app (connects via USB or WiFi)

CAMERA_SOURCE = "webcam"  # ← CHANGE THIS to "phone_ip" or "droidcam"

# 📱 Phone Camera Settings (for IP Webcam app - Android)
PHONE_IP = "172.24.148.110"  # ← Replace with YOUR phone's IP address
PHONE_PORT = "4747"         # Default port for IP Webcam app

# Alternative: DroidCam settings
DROIDCAM_IP = "172.24.148.110"  # ← Replace with your phone's IP
DROIDCAM_PORT = "4747"          # Default DroidCam port

# USB camera index (for webcam)
USB_CAMERA_INDEX = 0  # Try 0, 1, 2 if webcam doesn't work

def get_camera_url():
    """Returns the appropriate camera URL based on CAMERA_SOURCE setting."""
    if CAMERA_SOURCE == "phone_ip":
        return f"http://{PHONE_IP}:{PHONE_PORT}/video"
    elif CAMERA_SOURCE == "droidcam":
        return f"http://{DROIDCAM_IP}:{DROIDCAM_PORT}/video"
    else:
        return USB_CAMERA_INDEX

print("✓ Libraries imported successfully")
print(f"\n📷 Camera source: {CAMERA_SOURCE}")
if CAMERA_SOURCE == "phone_ip":
    print(f"   URL: http://{PHONE_IP}:{PHONE_PORT}/video")
    print("   📱 Make sure IP Webcam app is running on your phone!")
elif CAMERA_SOURCE == "droidcam":
    print(f"   URL: http://{DROIDCAM_IP}:{DROIDCAM_PORT}/video")
    print("   📱 Make sure DroidCam is running!")


✓ Libraries imported successfully

📷 Camera source: webcam


In [34]:
class OptimizedCamera:
    """
    Optimized camera capture with threading for better performance.
    Supports webcam, phone IP camera, and DroidCam.
    """
    def __init__(self, source, buffer_size=2):
        self.source = source
        self.cap = None
        self.frame_queue = queue.Queue(maxsize=buffer_size)
        self.running = False
        self.thread = None
        self.last_frame = None
        self.fps = 0
        self.frame_count = 0
        self.start_time = time.time()
        
    def start(self):
        """Start the camera capture thread."""
        self.cap = cv2.VideoCapture(self.source)
        
        if not self.cap.isOpened():
            raise ConnectionError(f"❌ Cannot connect to camera: {self.source}")
        
        # Optimize capture settings for USB webcam
        if isinstance(self.source, int):
            self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
            self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
            self.cap.set(cv2.CAP_PROP_FPS, 30)
            self.cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        
        self.running = True
        self.thread = Thread(target=self._capture_loop, daemon=True)
        self.thread.start()
        print(f"✓ Camera started: {self.source}")
        return self
    
    def _capture_loop(self):
        """Background thread for capturing frames."""
        while self.running:
            ret, frame = self.cap.read()
            if ret:
                self.last_frame = frame
                self.frame_count += 1
                
                elapsed = time.time() - self.start_time
                if elapsed > 1:
                    self.fps = self.frame_count / elapsed
                    self.frame_count = 0
                    self.start_time = time.time()
                
                if not self.frame_queue.full():
                    self.frame_queue.put(frame)
                else:
                    try:
                        self.frame_queue.get_nowait()
                        self.frame_queue.put(frame)
                    except queue.Empty:
                        pass
            else:
                time.sleep(0.001)
    
    def read(self):
        """Get the latest frame (non-blocking)."""
        if self.last_frame is not None:
            return True, self.last_frame.copy()
        return False, None
    
    def stop(self):
        """Stop the camera capture."""
        self.running = False
        if self.thread:
            self.thread.join(timeout=1.0)
        if self.cap:
            self.cap.release()
        print("✓ Camera stopped")
    
    def get_fps(self):
        return self.fps
    
    def isOpened(self):
        return self.cap is not None and self.cap.isOpened()


def test_camera_connection():
    """Test camera connection."""
    print("🔍 Testing camera connection...")
    camera_url = get_camera_url()
    print(f"   Connecting to: {camera_url}")
    
    try:
        cap = cv2.VideoCapture(camera_url)
        
        if not cap.isOpened():
            print("❌ Failed to connect!")
            if CAMERA_SOURCE == "phone_ip":
                print("\n📱 Troubleshooting for IP Webcam:")
                print("   1. Open IP Webcam app on your phone")
                print("   2. Scroll down and tap 'Start server'")
                print("   3. Note the IP address shown")
                print("   4. Update PHONE_IP in Cell 1")
                print("   5. Make sure phone and PC are on same WiFi")
            elif CAMERA_SOURCE == "droidcam":
                print("\n📱 Troubleshooting for DroidCam:")
                print("   1. Install DroidCam on phone AND PC")
                print("   2. Open app on phone, note the IP")
                print("   3. Update DROIDCAM_IP in Cell 1")
            return False
        
        print("✓ Camera connected!")
        for i in range(3):
            ret, frame = cap.read()
            if ret:
                print(f"   ✓ Frame {i+1}: {frame.shape}")
        
        cap.release()
        print("✓ Camera test complete!")
        return True
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return False

# Run the test
test_camera_connection()


🔍 Testing camera connection...
   Connecting to: 0
✓ Camera connected!
   ✓ Frame 1: (480, 640, 3)
   ✓ Frame 2: (480, 640, 3)
   ✓ Frame 3: (480, 640, 3)
✓ Camera test complete!


True

Model loading


In [35]:
# ============================================
# 🧠 MODEL LOADING (Optimized)
# ============================================

# Enable GPU memory growth to prevent OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✓ GPU configured: {len(gpus)} GPU(s) available")
    except RuntimeError as e:
        print(f"⚠️ GPU config error: {e}")
else:
    print("ℹ️ No GPU detected, using CPU")

# Load models with error handling
mobilenet_model = None
mlp_model = None

try:
    if os.path.exists("sign_language_model_MobileNetV2.h5"):
        mobilenet_model = tf.keras.models.load_model("sign_language_model_MobileNetV2.h5")
        print("✓ MobileNetV2 model loaded")
    else:
        print("⚠️ MobileNetV2 model not found (optional)")
except Exception as e:
    print(f"⚠️ MobileNetV2 load error: {e}")

try:
    if os.path.exists("asl_mediapipe_mlp_model.h5"):
        mlp_model = tf.keras.models.load_model("asl_mediapipe_mlp_model.h5")
        print("✓ MediaPipe MLP model loaded")
    else:
        raise FileNotFoundError("asl_mediapipe_mlp_model.h5 not found")
except Exception as e:
    print(f"❌ MLP model error: {e}")

# Load encoder
encoder = None
try:
    if os.path.exists("asl_mediapipe_keypoints_dataset.csv"):
        df = pd.read_csv("asl_mediapipe_keypoints_dataset.csv")
        encoder = LabelEncoder()
        encoder.fit(df["label"])
        del df  # Free memory
        print("✓ Encoder loaded")
except Exception as e:
    print(f"❌ Encoder error: {e}")

# Class labels
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J',
                'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T',
                'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']

if mlp_model is None or encoder is None:
    print("\n⚠️  Warning: Essential models failed to load. Camera detection will not work.")


✓ GPU configured: 1 GPU(s) available
✓ MobileNetV2 model loaded
✓ MediaPipe MLP model loaded
✓ Encoder loaded


Reading the mediapipe keypoints and defining the class labels


In [ ]:
# ============================================
# ⚙️ OPTIMIZED SETTINGS
# ============================================

# Display settings
DISPLAY_WIDTH = 1280
DISPLAY_HEIGHT = 720
WINDOW_RESIZABLE = True  # Allow window resizing

# Performance settings
PROCESS_EVERY_N_FRAMES = 1     # 1 = process all, 2 = skip every other frame
TARGET_FPS = 30                 # Target display FPS
SKIP_PREDICTION_NO_HAND = True  # Skip prediction when no hand detected

# MediaPipe settings (optimized for speed)
MP_MIN_DETECTION_CONFIDENCE = 0.6
MP_MIN_TRACKING_CONFIDENCE = 0.6
MP_MAX_HANDS = 1

# ============================================
# LETTER COMMIT STRATEGY (commit-once-then-wait)
# ============================================
# After a letter is accepted, the system WAITS until:
#   1. The hand disappears from the frame, OR
#   2. A DIFFERENT stable sign is held
# This completely prevents repeated letters like "mmmmmmooooooccccc"

STABILIZATION_WINDOW_SIZE = 10  # Buffer size for majority vote
STABILIZATION_THRESHOLD = 7    # Minimum matches in buffer (7/10 = 70% majority)
MIN_CONFIDENCE = 0.70          # Minimum confidence to consider a prediction
HOLD_TIME_REQUIRED = 0.8       # Seconds to hold sign before accepting

# Bounding box expansion (for MobileNet)
HEIGHT_EXPAND = 200
WIDTH_EXPAND = 130

print("⚙️ Settings configured:")
print(f"   Display: {DISPLAY_WIDTH}x{DISPLAY_HEIGHT} (Resizable: {WINDOW_RESIZABLE})")
print(f"   Target FPS: {TARGET_FPS}")
print(f"   Confidence threshold: {MIN_CONFIDENCE}")
print(f"   Stability: {STABILIZATION_THRESHOLD}/{STABILIZATION_WINDOW_SIZE} frames majority required")
print(f"   Hold time: {HOLD_TIME_REQUIRED}s before accepting")
print(f"   Strategy: commit-once, then wait for hand-drop or sign-change")


⚙️ Settings configured:
   Display: 1280x720 (Resizable: True)
   Target FPS: 30
   Confidence threshold: 0.92
   Cooldown time: 3.0s
   Buffer size: 7 frames
   Buffer threshold: 5 matches required
   Hold time: 0.5s


Mediapipe for hand keypoints


In [37]:
# Initialize MediaPipe with optimized settings
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=MP_MAX_HANDS,
    min_detection_confidence=MP_MIN_DETECTION_CONFIDENCE,
    min_tracking_confidence=MP_MIN_TRACKING_CONFIDENCE
)

print("✓ MediaPipe Hands initialized")
print(f"   - Detection confidence: {MP_MIN_DETECTION_CONFIDENCE}")
print(f"   - Tracking confidence: {MP_MIN_TRACKING_CONFIDENCE}")
print(f"   - Max hands: {MP_MAX_HANDS}")


✓ MediaPipe Hands initialized
   - Detection confidence: 0.6
   - Tracking confidence: 0.6
   - Max hands: 1


Define the height and width of the box, where mobilenet is to be predicting classes


In [ ]:
# Pre-allocate arrays for better performance
_landmark_buffer = np.zeros((21, 3), dtype=np.float32)

def extract_landmark_features_fast(hand_landmarks, is_right_hand=False):
    """Optimized landmark extraction with pre-allocated buffer.
    
    No mirroring is applied — training data was recorded without flipping,
    so inference landmarks must be used as-is regardless of handedness.
    """
    global _landmark_buffer
    
    for i, lm in enumerate(hand_landmarks.landmark):
        _landmark_buffer[i, 0] = lm.x
        _landmark_buffer[i, 1] = lm.y
        _landmark_buffer[i, 2] = lm.z
    
    return _landmark_buffer.flatten().reshape(1, -1)

def get_hand_bounding_box(hand_landmarks, frame_shape):
    """Get expanded bounding box for hand region."""
    h, w = frame_shape[:2]
    
    x_coords = [lm.x * w for lm in hand_landmarks.landmark]
    y_coords = [lm.y * h for lm in hand_landmarks.landmark]
    
    x_min = max(0, int(min(x_coords) - WIDTH_EXPAND))
    y_min = max(0, int(min(y_coords) - HEIGHT_EXPAND))
    x_max = min(w, int(max(x_coords) + WIDTH_EXPAND))
    y_max = min(h, int(max(y_coords) + HEIGHT_EXPAND))
    
    return x_min, y_min, x_max, y_max

# Keep original function for compatibility
def extract_landmark_features(hand_landmarks, handedness):
    """Original landmark extraction (for backward compatibility)."""
    is_right = handedness.classification[0].label == "Right"
    return extract_landmark_features_fast(hand_landmarks, is_right)

print("✓ Feature extraction functions defined (no mirroring — matches training data)")


✓ Feature extraction functions defined


Main Logic for comparision of confidence score of both models - MobileNet V2 and Mediapipe


In [ ]:
def run_sign_recognition():
    """Main sign language recognition loop — commit-once-then-wait strategy.
    
    How it works:
    1. Predictions go into a stability buffer (majority vote).
    2. Once a letter is stable + held long enough → it is committed ONCE.
    3. After committing, the system LOCKS that label and waits for either:
       a) The hand leaves the frame (resets everything), OR
       b) A DIFFERENT stable sign emerges (unlocks and starts tracking the new sign).
    This completely prevents "mmmmmooooooccccc" repetition.
    """
    
    if mlp_model is None or encoder is None:
        print("❌ Models not loaded. Cannot run recognition.")
        return
    
    # Initialize camera
    camera_url = get_camera_url()
    print(f"📷 Connecting to camera: {camera_url}")
    
    try:
        camera = OptimizedCamera(camera_url).start()
    except ConnectionError as e:
        print(e)
        return
    
    time.sleep(1.0)  # Wait for camera to warm up
    
    # State variables
    predicted_sentence = ""
    stabilization_buffer = deque(maxlen=STABILIZATION_WINDOW_SIZE)
    frame_count = 0
    fps_display = 0
    fps_start_time = time.time()
    
    # --- Commit-once-then-wait state ---
    committed_label = None       # The label we already committed (locked)
    current_sign_label = None    # The current stable sign being tracked
    current_sign_start = None    # When we first saw this stable sign
    waiting_for_change = False   # True after committing → waiting for hand-drop or new sign
    
    # Create resizable window
    window_name = "Sign Language Recognition"
    cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
    if WINDOW_RESIZABLE:
        cv2.resizeWindow(window_name, DISPLAY_WIDTH, DISPLAY_HEIGHT)
    
    print("\n" + "="*50)
    print("🤟 SIGN LANGUAGE RECOGNITION STARTED")
    print("="*50)
    print("Controls:")
    print("  'q' - Quit")
    print("  'c' - Clear sentence")
    print("  's' - Save screenshot")
    print("  Window is RESIZABLE - drag corners to resize!")
    print("="*50 + "\n")
    
    try:
        while True:
            ret, frame = camera.read()
            if not ret:
                continue
            
            frame_count += 1
            
            if PROCESS_EVERY_N_FRAMES > 1 and frame_count % PROCESS_EVERY_N_FRAMES != 0:
                continue
            
            # Process unflipped frame with MediaPipe (matches training data)
            frame = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_frame.flags.writeable = False
            results = hands.process(rgb_frame)
            rgb_frame.flags.writeable = True
            
            current_label = None
            current_confidence = 0
            display_status = ""
            status_color = (200, 200, 200)
            
            if results.multi_hand_landmarks:
                for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                    mp_drawing.draw_landmarks(
                        frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                        mp_drawing_styles.get_default_hand_landmarks_style(),
                        mp_drawing_styles.get_default_hand_connections_style()
                    )
                    
                    is_right = handedness.classification[0].label == "Right"
                    
                    # MLP Prediction
                    features = extract_landmark_features_fast(hand_landmarks, is_right)
                    mlp_pred = mlp_model.predict(features, verbose=0)
                    mlp_idx = np.argmax(mlp_pred)
                    mlp_conf = mlp_pred[0][mlp_idx]
                    mlp_label = encoder.inverse_transform([mlp_idx])[0]
                    
                    current_label = mlp_label
                    current_confidence = mlp_conf
                    
                    # MobileNet prediction (if model available)
                    if mobilenet_model is not None:
                        x1, y1, x2, y2 = get_hand_bounding_box(hand_landmarks, frame.shape)
                        hand_crop = frame[y1:y2, x1:x2]
                        
                        if hand_crop.size > 0:
                            hand_resized = cv2.resize(hand_crop, (128, 128))
                            hand_input = np.expand_dims(hand_resized, axis=0) / 255.0
                            
                            mob_pred = mobilenet_model.predict(hand_input, verbose=0)
                            mob_idx = np.argmax(mob_pred)
                            mob_conf = mob_pred[0][mob_idx]
                            mob_label = class_labels[mob_idx]
                            
                            if mob_conf > current_confidence:
                                current_label = mob_label
                                current_confidence = mob_conf
                        
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # --- COMMIT-ONCE-THEN-WAIT LOGIC ---
                    
                    # Skip low-confidence predictions entirely
                    if current_confidence < MIN_CONFIDENCE:
                        display_status = f"{current_label} ({current_confidence:.0%}) Low conf"
                        status_color = (0, 100, 255)  # Orange
                        break
                    
                    # Add to stability buffer
                    stabilization_buffer.append(current_label)
                    
                    # Check majority vote in buffer
                    buffer_count = stabilization_buffer.count(current_label)
                    is_stable = (buffer_count >= STABILIZATION_THRESHOLD and 
                                 len(stabilization_buffer) == STABILIZATION_WINDOW_SIZE)
                    
                    if not is_stable:
                        # Not stable yet — show progress
                        progress = buffer_count / STABILIZATION_THRESHOLD * 100
                        display_status = f"{current_label} ({current_confidence:.0%}) Stabilizing {progress:.0f}%"
                        status_color = (0, 255, 255)  # Yellow
                        break
                    
                    # Stable prediction achieved
                    now = time.time()
                    
                    # If we're waiting after a commit, check if the sign changed
                    if waiting_for_change:
                        if current_label == committed_label:
                            # Same sign still held — keep waiting
                            display_status = f"{current_label} ({current_confidence:.0%}) ✓ Committed - change sign for next"
                            status_color = (255, 200, 0)  # Cyan
                            break
                        else:
                            # Different sign! Unlock and start tracking the new one
                            waiting_for_change = False
                            committed_label = None
                            current_sign_label = current_label
                            current_sign_start = now
                    
                    # Track hold time for current sign
                    if current_label != current_sign_label:
                        # New sign — start hold timer
                        current_sign_label = current_label
                        current_sign_start = now
                    
                    hold_duration = now - current_sign_start if current_sign_start else 0
                    
                    if hold_duration < HOLD_TIME_REQUIRED:
                        # Still need to hold longer
                        hold_pct = hold_duration / HOLD_TIME_REQUIRED * 100
                        display_status = f"{current_label} ({current_confidence:.0%}) Hold: {hold_pct:.0f}%"
                        status_color = (0, 255, 255)  # Yellow
                        break
                    
                    # --- COMMIT THE LETTER ---
                    if current_label not in ["nothing", "del", "space"]:
                        predicted_sentence += current_label
                    elif current_label == "space":
                        if not predicted_sentence.endswith(" "):
                            predicted_sentence += " "
                    elif current_label == "del" and predicted_sentence:
                        predicted_sentence = predicted_sentence[:-1]
                    
                    committed_label = current_label
                    waiting_for_change = True
                    current_sign_label = None
                    current_sign_start = None
                    stabilization_buffer.clear()
                    
                    display_status = f"{current_label} ({current_confidence:.0%}) ✓ COMMITTED!"
                    status_color = (0, 255, 0)  # Green
            else:
                # No hand detected — FULL RESET (allows re-doing same letter)
                committed_label = None
                waiting_for_change = False
                current_sign_label = None
                current_sign_start = None
                stabilization_buffer.clear()
                display_status = "No hand detected"
                status_color = (150, 150, 150)
            
            # Flip for selfie-view display
            frame = cv2.flip(frame, 1)
            
            # Calculate FPS
            fps_elapsed = time.time() - fps_start_time
            if fps_elapsed >= 1.0:
                fps_display = frame_count / fps_elapsed
                frame_count = 0
                fps_start_time = time.time()
            
            # Draw UI elements
            # Top info bar
            cv2.rectangle(frame, (0, 0), (DISPLAY_WIDTH, 70), (30, 30, 30), -1)
            info_text = f"FPS: {fps_display:.1f} | Camera: {CAMERA_SOURCE} | 'q'=quit 'c'=clear 's'=screenshot"
            cv2.putText(frame, info_text, (10, 25), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 1)
            cv2.putText(frame, display_status, (10, 55), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.9, status_color, 2)
            
            # Bottom bar for sentence
            bar_height = 80
            cv2.rectangle(frame, (0, DISPLAY_HEIGHT - bar_height), 
                         (DISPLAY_WIDTH, DISPLAY_HEIGHT), (30, 30, 30), -1)
            cv2.putText(frame, predicted_sentence[-50:], (20, DISPLAY_HEIGHT - 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            
            cv2.imshow(window_name, frame)
            
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('c'):
                predicted_sentence = ""
                committed_label = None
                waiting_for_change = False
                stabilization_buffer.clear()
                print("🗑️ Sentence cleared")
            elif key == ord('s'):
                filename = f"screenshot_{int(time.time())}.png"
                cv2.imwrite(filename, frame)
                print(f"📸 Screenshot saved: {filename}")
    
    except KeyboardInterrupt:
        print("\n⚠️ Interrupted by user")
    
    finally:
        camera.stop()
        cv2.destroyAllWindows()
        print(f"\n📝 Final sentence: {predicted_sentence}")

# Run the recognition
run_sign_recognition()


IndentationError: unexpected indent (4036012915.py, line 209)

From the experiment above it can be concluded that models based on convolutional neural networks aren't able to predict the signs correctly and even when combined with predictions of mediapipe multi-level perceptron, it still gives the wrong output.


Therefore fine tuning the mediapipe will be the best option available in the market.


Fine tuning the Mediapipe Model


In [40]:
# Error handling for model loading (Fine-tuned version)
try:
    if os.path.exists("asl_mediapipe_mlp_model.h5"):
        mlp_model = tf.keras.models.load_model("asl_mediapipe_mlp_model.h5")
        print("✓ MediaPipe MLP model loaded successfully")
    else:
        raise FileNotFoundError("asl_mediapipe_mlp_model.h5 not found")
except Exception as e:
    print(f"❌ Error loading MediaPipe MLP model: {e}")
    mlp_model = None
    print("⚠️  Warning: Model failed to load. Camera detection will not work.")


✓ MediaPipe MLP model loaded successfully


In [41]:
# Error handling for CSV file (Fine-tuned version)
try:
    if os.path.exists("asl_mediapipe_keypoints_dataset.csv"):
        df = pd.read_csv("asl_mediapipe_keypoints_dataset.csv")
        encoder = LabelEncoder()
        encoder.fit(df["label"])
        print("✓ CSV file loaded and encoder fitted successfully")
    else:
        raise FileNotFoundError("asl_mediapipe_keypoints_dataset.csv not found")
except Exception as e:
    print(f"❌ Error loading CSV file: {e}")
    encoder = None
    print("⚠️  Warning: Encoder failed to load. Camera detection will not work.") 


✓ CSV file loaded and encoder fitted successfully


In [42]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7, max_num_hands=2)  # Allow max 2 hands


In [43]:
# Sentence formation logic
predicted_sentence = ""
last_predicted_label = None
last_prediction_time = 0

# 5 seconds cooldown for repeated letters
cooldown_time = 5 


In [44]:
# Optimized Stabilization buffer - SLOWED DOWN
# Stores last 7 predictions (increased for better stability)
stabilization_window = deque(maxlen=7)

# Must match for 5 out of 7 frames (stricter requirement)
stabilization_threshold = 5

# Confidence threshold - only very high confidence can bypass buffer
min_confidence = 0.92  # Increased from 0.85

# Cooldown for repeated letters
cooldown_time = 3  # Increased from 5 to 3 (but combined with hold time)

two_hands_detected = False

print("⏱️ Buffer settings (slowed down):")
print(f"   - Buffer size: {stabilization_window.maxlen} frames")
print(f"   - Required matches: {stabilization_threshold}")
print(f"   - Min confidence: {min_confidence}")


⏱️ Buffer settings (slowed down):
   - Buffer size: 7 frames
   - Required matches: 5
   - Min confidence: 0.92


In [ ]:
def extract_landmark_features(hand_landmarks, handedness):
    """Extract and normalize 21 hand landmarks from MediaPipe.
    
    No mirroring is applied — training data was recorded without flipping,
    so inference landmarks must be used as-is.
    """
    landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
    return landmarks.flatten().reshape(1, -1)


What is fine tuned


- Making of a buffer ensuring that model doesnt predicts images in between the hand sign change
- only predic when there is one hand in the frame, when 2 hand appear, give an warning


In [ ]:
# Fine-Tuned MLP-Only Camera — commit-once-then-wait strategy
if mlp_model is None or encoder is None:
    print("❌ Error: Models not loaded. Please check Cell 14 and Cell 15 for errors.")
else:
    camera_url = get_camera_url()
    cap = cv2.VideoCapture(camera_url)
    
    if not cap.isOpened():
        print("❌ Error: Cannot access camera.")
    else:
        print("✓ Camera opened successfully")
        print("Press 'q' to quit, 'c' to clear | Window is RESIZABLE")
        
        window_name = "Sign Language Recognition (Fine Tuned MediaPipe)"
        cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
        cv2.resizeWindow(window_name, DISPLAY_WIDTH, DISPLAY_HEIGHT)
        
        # State variables
        predicted_sentence = ""
        stabilization_buffer = deque(maxlen=STABILIZATION_WINDOW_SIZE)
        two_hands_detected = False
        
        # Commit-once-then-wait state
        committed_label = None
        current_sign_label = None
        current_sign_start = None
        waiting_for_change = False
        
        try:
            while cap.isOpened():
                ret, frame = cap.read()
                if not ret:
                    break

                # Process unflipped frame for MediaPipe (matches training data)
                frame = cv2.resize(frame, (DISPLAY_WIDTH, DISPLAY_HEIGHT))
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                rgb_frame.flags.writeable = False
                results = hands.process(rgb_frame)
                rgb_frame.flags.writeable = True

                two_hands_detected = (results.multi_hand_landmarks and 
                                      len(results.multi_hand_landmarks) > 1)

                display_status = ""
                status_color = (200, 200, 200)

                if results.multi_hand_landmarks and not two_hands_detected:
                    for hand_landmarks, handedness in zip(results.multi_hand_landmarks, results.multi_handedness):
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                        # MLP prediction
                        landmark_features = extract_landmark_features(hand_landmarks, handedness)
                        mlp_pred = mlp_model.predict(landmark_features, verbose=0)
                        mlp_class_index = np.argmax(mlp_pred)
                        mlp_confidence = mlp_pred[0][mlp_class_index]
                        mlp_label = encoder.inverse_transform([mlp_class_index])[0]

                        # Skip low confidence
                        if mlp_confidence < MIN_CONFIDENCE:
                            display_status = f"{mlp_label} ({mlp_confidence:.0%}) Low conf"
                            status_color = (0, 100, 255)
                            break

                        # Add to stability buffer
                        stabilization_buffer.append(mlp_label)

                        buffer_count = stabilization_buffer.count(mlp_label)
                        is_stable = (buffer_count >= STABILIZATION_THRESHOLD and 
                                     len(stabilization_buffer) == STABILIZATION_WINDOW_SIZE)

                        if not is_stable:
                            progress = buffer_count / STABILIZATION_THRESHOLD * 100
                            display_status = f"{mlp_label} ({mlp_confidence:.0%}) Stabilizing {progress:.0f}%"
                            status_color = (0, 255, 255)
                            break

                        now = time.time()

                        # Check if waiting after a commit
                        if waiting_for_change:
                            if mlp_label == committed_label:
                                display_status = f"{mlp_label} ({mlp_confidence:.0%}) ✓ Committed - change sign"
                                status_color = (255, 200, 0)
                                break
                            else:
                                waiting_for_change = False
                                committed_label = None
                                current_sign_label = mlp_label
                                current_sign_start = now

                        # Track hold time
                        if mlp_label != current_sign_label:
                            current_sign_label = mlp_label
                            current_sign_start = now

                        hold_duration = now - current_sign_start if current_sign_start else 0

                        if hold_duration < HOLD_TIME_REQUIRED:
                            hold_pct = hold_duration / HOLD_TIME_REQUIRED * 100
                            display_status = f"{mlp_label} ({mlp_confidence:.0%}) Hold: {hold_pct:.0f}%"
                            status_color = (0, 255, 255)
                            break

                        # COMMIT
                        if mlp_label not in ["nothing", "del", "space"]:
                            predicted_sentence += mlp_label
                        elif mlp_label == "space":
                            if not predicted_sentence.endswith(" "):
                                predicted_sentence += " "
                        elif mlp_label == "del" and predicted_sentence:
                            predicted_sentence = predicted_sentence[:-1]

                        committed_label = mlp_label
                        waiting_for_change = True
                        current_sign_label = None
                        current_sign_start = None
                        stabilization_buffer.clear()

                        display_status = f"{mlp_label} ({mlp_confidence:.0%}) ✓ COMMITTED!"
                        status_color = (0, 255, 0)
                else:
                    # No hand → full reset
                    committed_label = None
                    waiting_for_change = False
                    current_sign_label = None
                    current_sign_start = None
                    stabilization_buffer.clear()

                # Flip for selfie-view display
                frame = cv2.flip(frame, 1)

                # Status text
                cv2.putText(frame, display_status, (10, 30), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, status_color, 2)

                # Bottom bar
                bar_height = 60
                frame_height, frame_width, _ = frame.shape
                cv2.rectangle(frame, (0, frame_height - bar_height), 
                             (frame_width, frame_height), (0, 0, 0), -1)

                if two_hands_detected:
                    cv2.putText(frame, "Only One Hand Allowed!", (50, frame_height - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                else:
                    cv2.putText(frame, predicted_sentence[-50:], (50, frame_height - 20),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

                cv2.imshow(window_name, frame)
                key = cv2.waitKey(1) & 0xFF
                if key == ord('q'):
                    break
                elif key == ord('c'):
                    predicted_sentence = ""
                    committed_label = None
                    waiting_for_change = False
                    stabilization_buffer.clear()
                    print("🗑️ Sentence cleared")

        except KeyboardInterrupt:
            print("\n⚠️ Interrupted by user")
        except Exception as e:
            print(f"❌ Error during camera loop: {e}")
        finally:
            cap.release()
            cv2.destroyAllWindows()
            print(f"📝 Final sentence: {predicted_sentence}")


✓ Camera opened successfully
Press 'q' to quit (or Interrupt in Jupyter)
🪟 Window is RESIZABLE - drag corners to resize!
✓ Camera released
